In [275]:
from api_key import get_api_key_jef, get_api_key_sha, get_api_key_sup
import google.generativeai as genai
from vpn_control import windscribe
import time
import re
import pandas as pd
import numpy as np
import csv
import os


In [276]:
windscribe("connect", "Atlanta")
time.sleep(5)

In [277]:
passw = "2002"
LLM_MODEL = "gemini-1.5-flash" #gemini-1.5-pro-latest

In [278]:
def setup_prompt_env(passw, extra_instruct):


  genai.configure(api_key=get_api_key_sup(passw))

  # Set up the model
  generation_config = {
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 3000,
  }

  safety_settings = [
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
  ]

  system_instruction = f"""You must modify all the wildcards <*> with varibles which would make sense in that conxtext such as time, IP address etc. Additonally {extra_instruct}. Pad each log with a begin and end token <B> <E>. For each input create 36 logs with different varibles filled into the wildcards <*>. Find two examples below
  
  INPUT: PacketResponder <*> for block blk_<*> terminating
  OUTPUT: 
    [1] <B>PacketResponder: PacketResponder 1 for block blk_38865049064139660 terminating<E>
    [2] <B>PacketResponder: PacketResponder 0 for block blk_-6952295868487656571 terminating<E>
    ...
    [36] <B>PacketResponder: PacketResponder 1 for block blk_509586258217225674 terminating<E>

  INPUT: BLOCK* NameSystem.addStoredBlock: blockMap updated: <*>:<*> is added to blk_<*> size <*>
  OUTPUT: 
    [1] <B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.250.11.85:50010 is added to blk_2377150260128098806 size 67108864<E>
    [2] <B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.203.80:50010 is added to blk_7888946331804732825 size 67108864<E>
    ...
    [36] <B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.250.11.85:50010 is added to blk_2377150260128098806 size 67108864<E>

  
 Be creative with numbers DO NOT use repeating or ordered numbers such as 1111,2222 or 1234 DO NOT repeat numbers; use IP address to specify nodes instead of path. Do NOT change the rest of the log, only <*> and DO NOT RETURN ANY extra text."""

  model = genai.GenerativeModel(model_name=LLM_MODEL,
                                generation_config=generation_config,
                                system_instruction=system_instruction,
                                safety_settings=safety_settings)

  convo = model.start_chat(history=[
  ])

  return convo


In [279]:
def create_logs_LLM(message, convo, extra_instruct):
    message =  message 
    convo.send_message(message)
    suggestion = convo.last.text
    #print(suggestion)
    return suggestion

In [280]:

def setup_LLM_logs(passw, extra_instruct, message):
    convo_create = setup_prompt_env(passw, extra_instruct)
    res = create_logs_LLM(message, convo_create, extra_instruct)
    return res



In [281]:
#setup_LLM_logs(passw, ".", "BLOCK* NameSystem.addStoredBlock: blockMap updated: <*>:<*> is added to blk_<*> size <*>")
# #BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.43.115:50010 is added to blk_3050920587428079149 size 67108864

In [282]:
# setup_LLM_logs(passw, ".", "Received block blk_<*> of size <*> from /<*>")
# #Received block blk_3587508140051953248 of size 67108864 from /10.251.42.84

In [283]:
tmep = pd.read_csv("data/augmented_templates_flash.csv")
df = tmep
df

,EventId,EventTemplate,0,1,2,3,4
0,E1,<*>:<*> Served block blk_<*> to /<*>,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B><*>:<*> Delivered block blk_<*> to /<*> <E>,<B>Block blk_<*> served by <*> to /<*> <E>,<B>Block blk_<*> was delivered to /<*> by <*> <E>,<B>Successful block delivery of blk_<*> to /<*...
1,E2,<*>:<*> Starting thread to transfer block blk_...,<B><*>:<*> Starting thread to transfer block b...,<B>Initiating transfer of block blk_<*> from <...,<B>Transfer thread started for block blk_<*> f...,<B>A thread has been started to transfer block...,<B>Block blk_<*> transfer initiated from <*>:<...
2,E3,<*>:<*>:Got exception while serving blk_<*> to...,<B><*>:<*>:Got exception while serving blk_<*>...,<B><*>:<*>:Exception occurred while serving bl...,<B>Exception encountered on <*> while serving ...,<B>Block <*> could not be served to /<*> on <*...,<B>Error: Block <*> could not be delivered to ...
3,E4,BLOCK* ask <*>:<*> to delete blk_<*>,<B>BLOCK* ask <*>:<*> to delete blk_<*><E>,<B>BLOCK* requests <*>:<*> to delete block blk...,<B>BLOCK* requests deletion of block blk_<*> f...,<B>Deletion request for block blk_<*> sent to ...,<B>BLOCK* initiated a block deletion request f...
4,E5,BLOCK* ask <*>:<*> to replicate blk_<*> to dat...,<B>BLOCK* ask <*>:<*> to replicate blk_<*> to ...,<B>BLOCK* request to replicate block blk_<*> f...,<B>Replication request for block blk_<*> from ...,<B>BLOCK* received a request to replicate bloc...,<B>Data replication task initiated for block b...
5,E6,BLOCK* NameSystem.addStoredBlock: blockMap upd...,<B>BLOCK* NameSystem.addStoredBlock: blockMap ...,<B>BLOCK* NameSystem.addStoredBlock: blockMap ...,<B>BLOCK* NameSystem.addStoredBlock: blockMap ...,<B>NameSystem added block <*>:<*> to blk_<*> w...,<B>Block <*>:<*> with size <*> was added to bl...
6,E7,BLOCK* NameSystem.allocateBlock: /<*>/part-<*>...,<B>BLOCK* NameSystem.allocateBlock: /<*>/part-...,<B>NameSystem allocated block: /<*>/part-<*>. ...,<B>Block allocation: /<*>/part-<*>. blk_<*> by...,<B>NameSystem allocated block /<*>/part-<*> wi...,<B>Block allocation successful for /<*>/part-<...
7,E8,BLOCK* NameSystem.delete: blk_<*> is added to ...,<B>BLOCK* NameSystem.delete: blk_<*> is added ...,<B>Block <*> was added to the invalid set of <...,<B>NameSystem.delete added block <*> to the in...,<B>The NameSystem.delete operation added block...,<B>Block <*> was marked as invalid by NameSyst...
8,E9,Deleting block blk_<*> file /<*>/blk_<*>,<B>Deleting block blk_<*> file /<*>/blk_<*> <E>,<B>Deleting block file blk_<*> located at /<*>...,<B>Block file blk_<*> at /<*>/blk_<*> is being...,<B>File blk_<*> located at /<*>/blk_<*> is bei...,<B>Removal of block file blk_<*> from the bloc...
9,E10,PacketResponder <*> for block blk_<*> terminating,<B>PacketResponder <*> for block blk_<*> termi...,<B>PacketResponder <*> terminating for block b...,<B>PacketResponder <*> is terminating for bloc...,<B>Block blk_<*> is being terminated by Packet...,<B>PacketResponder <*> has stopped processing ...


In [284]:
def generate_and_store_logs(df, column_name, output_folder):
    if column_name not in df.columns:
        raise ValueError(f"Column {column_name} does not exist in the DataFrame")

    os.makedirs(output_folder, exist_ok=True)

    output_csv = os.path.join(output_folder, f'{column_name}_generated_logs.csv')
    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = ['EventId', 'LogTemplate', 'Log']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for index, row in df.iterrows():
            time.sleep(5)
            event_id = row['EventId']
            log_template = row[column_name]
            generated_logs = setup_LLM_logs(passw, ".", log_template).strip().split('\n')
            for log in generated_logs:
                writer.writerow({'EventId': event_id, 'LogTemplate': log_template, 'Log': log})

In [285]:
generate_and_store_logs(df, "0", "data/aug_templates")

In [289]:
time.sleep(30)
generate_and_store_logs(df, "1", "data/aug_templates")

In [290]:
time.sleep(100)
generate_and_store_logs(df, "2", "data/aug_templates")

In [291]:
time.sleep(100)
generate_and_store_logs(df, "3", "data/aug_templates")

In [292]:
time.sleep(100)
generate_and_store_logs(df, "4", "data/aug_templates")

In [293]:
#create func that checks only <*> has been replaced

In [294]:
# append
# 081109 203615 148 INFO dfs.DataNode$ 
# 081109 203807 222 INFO dfs.DataNode$
# 081109 204132 26 INFO dfs.FSNamesystem:
#081109 204655 556 INFO dfs.DataNode$PacketResponder: 

In [295]:
#windscribe("disconnect")